# A tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!


In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith("sk-proj-") and len(api_key)>10:
    print("API key is valid")
else:
    print("API key is invalid. Please chech your .env file")


MODEL = "gpt-4o-mini"
openai = OpenAI()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
}

API key is valid


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt = f"""You are an expert AI assistant specialized in answering technical questions with clear, detailed, and accurate explanations. 
Your goal is to provide responses that are:
- **Accurate**: Ensure factual correctness in your explanations.
- **Concise yet Detailed**: Keep responses structured and to the point while covering important details.
- **Beginner-Friendly but Technical**: Adapt explanations based on the complexity of the question.
- **Example-Driven**: Whenever possible, provide real-world examples or code snippets to clarify concepts.

Respond in a structured format:
1️⃣ **Definition**: Start with a brief explanation of the concept.
2️⃣ **How it Works**: Explain its functionality or underlying principles.
3️⃣ **Use Cases**: Describe where and how it is used.
4️⃣ **Example (if applicable)**: Provide a relevant example or code snippet.
5️⃣ **Common Mistakes & Best Practices**: Highlight any pitfalls or recommendations.

If a question is ambiguous, ask for clarification before answering.
If the question is outside of your scope, politely indicate that and suggest possible sources for information.
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(
    model = MODEL,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
        ],
    stream = True
)

response = ""
display_handle = display(Markdown(""), display_id = True)
for chunk in stream:
    response +=chunk.choices[0].delta.content or ""
    response = response.replace("``","").replace("markdown", "")
    update_display(Markdown(response), display_id = display_handle.display_id)


In [ ]:
# Get Llama 3.2 to answer

In [12]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODELOLL = "llama3.2"

payload = {
    "model": MODELOLL,
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
}

In [16]:
stream = requests.post(OLLAMA_API, headers=HEADERS, json = payload, stream = True)

response = ""
display_handle = display(Markdown(""), display_id = True)
for chunk in stream.iter_lines(decode_unicode=True):
    if chunk.strip():
        try:
            data = json.loads(chunk)
            
            response += data["message"]["content"]

            response = response.replace("``","").replace("markdown", "")
            update_display(Markdown(response), display_id = display_handle.display_id)
        except json.JSONDecodeError as e:
            print(f"Warning: Skipped malformed JSON chunk: {chunk} - Error: {e}")



**Definition**: `yield from` is a Python syntax introduced in version 3.3, which allows you to delegate iteration over a sub-iterator.

**How it Works**: In this code snippet, we're using `yield from` to iterate over a list of books and extract the author information from each book. The expression `{book.get("author") for book in books if book.get("author")}` is an iterator that yields values on the fly, without storing them in memory.

Here's a step-by-step breakdown:

1. `for book in books`: This part iterates over a list of `books`.
2. `if book.get("author")`: For each book, this condition checks if the "author" key exists and has a non-empty value.
3. `{book.get("author") for ...}`: If the condition is met, the dictionary's "author" value (or `None` if it doesn't exist) is yielded.

**Use Cases**: This syntax is commonly used when working with data structures that are too large to fit into memory or when you need to process a large dataset in chunks. It's particularly useful for:

* Reading large files or datasets
* Handling infinite iterables (e.g., generator expressions)
* Improving memory efficiency

**Example**:
`python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

for author in yield from {book.get("author") for book in books if book.get("author")]:
    print(author)
`
Output:
`
Author A
None
Author C
`

**Common Mistakes & Best Practices**: When using `yield from`, ensure that the sub-iterator is properly initialized and that you handle any potential errors. Additionally, be mindful of memory usage when working with large datasets.

**Additional Advice**:

* If you're familiar with generators, you might recognize this syntax as equivalent to a generator expression.
* To iterate over the yielded values instead of using `yield from`, you can use the `.next()` method or a `for` loop.
`python
author_iterator = {book.get("author") for book in books if book.get("author")}
while True:
    try:
        author = next(author_iterator)
        print(author)
    except StopIteration:
        break
`
This approach can be useful when you need to iterate over the yielded values after they've been generated.